## Homework 04 Alex Khvatov

In [13]:
!pip freeze | grep scikit-learn

In [2]:
!python -V

Python 3.12.10


In [21]:
!pwd

/home/alex/python_projects/course_mlops/homework4


In [22]:
import pickle
import pandas as pd
import numpy as np
from pathlib import Path

In [24]:
year = 2023
month = 3
output_file = Path('.') / 'output' / f'yellow_tripdata_predictions_{year:04d}-{month:02d}.parquet'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [17]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [18]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1. Notebook

We'll start with the same notebook we ended up with in homework 1. We cleaned it a little bit and kept only the scoring part. You can find the initial notebook here.

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

__6.24__

In [19]:
std_dev = np.std(y_pred)
std_dev

np.float64(6.247488852238703)

In [37]:
mean = np.mean(y_pred)
print(f'{mean:.3f} mean')

14.204 mean


### Q2. Preparing the output
Like in the course videos, we want to prepare the dataframe with the output.

First, let's create an artificial ride_id column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results.

Save it as parquet:
```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```
What's the size of the output file?


In [20]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [27]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction']= y_pred

In [28]:
df_result

,ride_id,prediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [29]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

__66M__

#### Q3. Creating the scoring script

```bash
jupyter nbconvert --to script homework.ipynb
```

#### Q4. Virtual environment
Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter notebook.

After installing the libraries, pipenv creates two files: Pipfile and Pipfile.lock. The Pipfile.lock file keeps the hashes of the dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

__"sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c",__


#### Q5. Parametrize the script
Let's now make the script configurable via CLI. We'll create two parameters: year and month.

Run the script for April 2023.

What's the mean predicted duration?

__Mean 14.292__

### Q6. Docker container
Finally, we'll package the script in the docker container. For that, you'll need to use a base image that we prepared.

This is what the content of this image is:
```docker
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed to `agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`, which you need to use as your base image.

That is, your Dockerfile should start with:
```docker
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```


This image already has a pickle file with a dictionary vectorizer and a model. You will need to use them.

_Important_: don't copy the model to the docker image. You will need to use the pickle file already in the image.

Now run the script with docker. What's the mean predicted duration for May 2023?

__Mean 0.192__